# Feature engineering and ETL

In the initial data exploration it was established that the data is skewed , has some outliers and missing values for numerical columns. There are categorical features that need to be one hot encoded. Since multiple models will be used multiple saves will be created depending on the transformations. For eg, Tree based models do not require that data be scaled while it is a good idea to scale data for neural networks.

In total 3 different ETL pipelines are created. One with one hot encoded categorical columns, one with scaled columns and one with scaled and transformed. The Yeo jhonson transformation is used on numerical columns to handle skewness. this should work well with Neural Network.

In [87]:
#imports
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

In [105]:
pd.set_option('display.max_info_columns',1)
path='/Users/nikk/Downloads/IBM Capstone/'
name='online_shoppers_intention.csv'
df=pd.read_csv(path+name)
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0.0,-1.0,0.0,-1.0,1.0,-1.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0.0,0.0,0.0,0.0,2.0,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0.0,0.0,0.0,0.0,10.0,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


As seen in the initial data exploration we clip the outlier for individual columns with the following upper and lower thresholds an also drop the missing values (0.1 percent are missing)

In [89]:
df['Administrative']=df['Administrative'].clip(lower=0,upper=20)
df['PageValues']=df['PageValues'].clip(lower=0,upper=250)
df['Administrative_Duration']=df['Administrative_Duration'].clip(lower=0,upper=1500)
df['Informational']=df['Informational'].clip(lower=0,upper=10)
df['ProductRelated_Duration']=df['ProductRelated_Duration'].clip(lower=0,upper=20000)
df['ProductRelated']=df['ProductRelated'].clip(lower=0,upper=400)
df['Informational_Duration']=df['Informational_Duration'].clip(lower=0,upper=1500)

VisitorType and Month columns in the datafram are objects or strings. We need a numerical represention for them

In [90]:
cat_dtype = pd.api.types.CategoricalDtype(categories=['Feb', 'Mar', 'May', 'June', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov','Dec'],
                                          ordered=True)#ordered dtype for month
df['Month']=df['Month'].astype(cat_dtype)

In [91]:
df.dtypes

Administrative              float64
Administrative_Duration     float64
Informational               float64
Informational_Duration      float64
ProductRelated              float64
ProductRelated_Duration     float64
BounceRates                 float64
ExitRates                   float64
PageValues                  float64
SpecialDay                  float64
Month                      category
OperatingSystems              int64
Browser                       int64
Region                        int64
TrafficType                   int64
VisitorType                  object
Weekend                        bool
Revenue                        bool
dtype: object

We convert all categorical columns to type category. The pandas functions get_dummies will convert the columns of type category to one hot encoded vector 

In [92]:
cat_cols=['Weekend','VisitorType','Region','Browser','OperatingSystems','TrafficType']
df[cat_cols]=df[cat_cols].astype('category')
df =df.dropna()

In [93]:
df_encoded=pd.get_dummies(df,columns=cat_cols+['Month'],prefix=cat_cols+['Month'])

Select the numeric data type so that they can be scaled

In [95]:
numeric_cols = df.select_dtypes(include='float64').columns

In [96]:
scaler = MinMaxScaler()
pt=PowerTransformer()
df_encoded_transformed = df_encoded.copy()
df_encoded_scaled = df_encoded.copy()
df_encoded_scaled[numeric_cols]= scaler.fit_transform(df_encoded[numeric_cols])
df_encoded_transformed[numeric_cols]= pt.fit_transform(df_encoded[numeric_cols])

In [97]:
save_name='online_shoppers_intention_encoded.csv'
df_encoded.to_csv(path+save_name,index=False)
save_name='online_shoppers_intention_encoded_scaled.csv'
df_encoded_scaled.to_csv(path+save_name,index=False)
save_name='online_shoppers_intention_encoded_scaled_transformed.csv'
df_encoded_transformed.to_csv(path+save_name,index=False)

Finally the state of the scaler and transformer are saved so that it can be used to rescale if necessary.

In [98]:
import pickle

In [99]:
pickle.dump(scaler,open('Min_Max_scaler.pkl','wb'))
pickle.dump(pt,open('PowerTRansformer.pkl','wb'))